# 初期設定

`conda activate llmAgent`

python_version = 3.11

`pip install openai`

`pip install python-dotenv`

`pip install rich`

# APIで色々遊んでみる

In [1]:
from dotenv import load_dotenv
load_dotenv()

import os 
OPENAI_APIKEY = os.getenv("OPENAI_APIKEY")
os.environ["OPENAI_API_KEY"] = OPENAI_APIKEY

In [2]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    temperature=0.0,
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "こんにちは"}],
)

print(response.choices[0].message.content)

こんにちは！どういったことをお手伝いできますか？


In [3]:
response

ChatCompletion(id='chatcmpl-BhrJEDoOwVBG0TcVE8cnt2E9iWKmu', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='こんにちは！どういったことをお手伝いできますか？', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1749792724, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_34a54ae93c', usage=CompletionUsage(completion_tokens=14, prompt_tokens=8, total_tokens=22, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [4]:
from rich import print_json
print_json(data=response.model_dump(), highlight=False)

{
  "id": "chatcmpl-BhrJEDoOwVBG0TcVE8cnt2E9iWKmu",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "こんにちは！どういったことをお手伝いできますか？",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1749792724,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_34a54ae93c",
  "usage": {
    "completion_tokens": 14,
    "prompt_tokens": 8,
    "total_tokens": 22,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}

In [5]:
history = []
n = 0 #会話回数の上限
model = "gpt-4o-mini"
for _ in range(n):
    user_input = input("ユーザ入力: ")
    if user_input == "exit":
        break
    print(f"ユーザ： {user_input}")
    history.append({"role": "user", "content": user_input})
    response = client.chat.completions.create(model=model, messages=history)
    content = response.choices[0].message.content
    print(f"AI: {content}")
    history.append({"role": "assistant", "content": content})

modelが出力する文章を一気に最後見るのではなく，徐々に見るように変える

In [8]:
history = []
n = 10 #会話回数の上限
model = "gpt-4o-mini"
for _ in range(n):
    user_input = input("ユーザ入力: ")
    if user_input == "exit":
        break
    print(f"ユーザ： {user_input}")
    history.append({"role": "user", "content": user_input})
    stream = client.chat.completions.create(model=model, messages=history, stream=True)
    print(f"AI: ", end="")
    ai_content = ""
    for chunk in stream:
        content = chunk.choices[0].delta.content
        if chunk.choices[0].finish_reason == "stop":
            break
        print(content, end="")
        ai_content += content
    print()
    history.append({"role": "assistant", "content": ai_content})

ユーザ： 強化学習について教えてください
AI: 強化学習（きょうかがくしゅう、Reinforcement Learning）は、機械学習の一分野で、エージェント（学習者）が環境と相互作用しながら最適な行動を学ぶアルゴリズムです。強化学習の目的は、長期的な報酬を最大化するための戦略（ポリシー）を学ぶことです。

以下に強化学習の基本的な要素と概念を説明します。

1. **エージェントと環境**:
   - **エージェント**: 学習を行う存在で、行動を選択します。
   - **環境**: エージェントが相互作用する外部の世界で、エージェントの行動に対する反応（報酬や新しい状態）を提供します。

2. **状態（State）**:
   - 環境の状況のことを指します。エージェントは現在の状態に基づいて行動を選択します。

3. **行動（Action）**:
   - エージェントが時間の中で選択できる可能性のあるアクションです。行動を選ぶことで、エージェントは環境に影響を与えます。

4. **報酬（Reward）**:
   - エージェントがある行動をとった結果得られるフィードバックのことです。短期的な評価となりますが、報酬が良ければその行動を強化したり、悪ければ避けるように学習します。

5. **ポリシー（Policy）**:
   - エージェントが特定の状態でどのような行動を選択するかを決定する戦略のことです。ポリシーは定常的（固定）または確率的に設定されることがあります。

6. **価値関数（Value Function）**:
   - 特定の状態や行動に対して期待される将来の報酬の合計を評価します。これによりエージェントは、どの状態やアクションが価値があるかを学習します。

強化学習は、ゲームプレイ（例えば、チェスや囲碁）、ロボティクス、ナビゲーション、資源管理など、さまざまな分野で利用されています。最近では、深層強化学習（Deep Reinforcement Learning）と呼ばれる手法もあり、深層学習を組み合わせることで複雑な問題に対する性能を向上させています。

強化学習は、試行錯誤を通じて学ぶため、データの収集と学習に時間がかかることがありますが、非常に柔軟で強力な学習方法です。


In [ ]:
# 画像を入力にしてみる
import base64
from pathlib import Path
from typing import Any

client = OpenAI()

def image2content(image_path: Path) -> dict[str,Any]:
    #base64エンコード
    with image_path.open("rb") as f:
        image_base64 = base64.b64encode(f.read().decode("utf-8"))
    
    content = {"type:": }